---

Notebook ini digunakan untuk proses cleaning data hasil scraping

Proses cleaning dilakukan untuk membersihkan data hasil scraping, agar data lebih relable untuk digunkan untuk proses selanjutnya (analysis dan model).

---

# Import Libaries

In [55]:
# Import Libraries

import json
import pandas as pd
import numpy as np
import re
import ast
from langdetect import detect, DetectorFactory
import warnings
warnings.filterwarnings('ignore')


# Load Data

Data file json hasil scraping di load dan digabungkan, kemudian disave sebagai backup.

In [56]:
# Read file JSON
with open('data-scrap/amazfit.json') as f1, open('data-scrap/apple.json') as f2, open('data-scrap/fitbit.json') as f3, open('data-scrap/garmin.json') as f4, open('data-scrap/google.json') as f5, open('data-scrap/huawei.json') as f6, open('data-scrap/samsung.json') as f7, open('data-scrap/xiaomi.json') as f8:
    data1 = json.load(f1)
    data2 = json.load(f2)
    data3 = json.load(f3)
    data4 = json.load(f4)
    data5 = json.load(f5)
    data6 = json.load(f6)
    data7 = json.load(f7)
    data8 = json.load(f8)

# Combine
combine = data1 + data2 + data3 + data4 + data5 + data6 + data7 + data8

# Simpan hasilnya ke file baru
with open('all.json', 'w') as outfile:
    json.dump(combine, outfile, indent=2, ensure_ascii=True)

In [57]:
# Read file JSON
with open('all.json', 'r') as file:
    data = json.load(file)

# To DataFrame
df = pd.DataFrame(data)

# Show DataFrame
df


,title,rating,price,features,Battery Capacity,Connectivity,GPS,Screen Size,img_url,Brand,Review
0,"Amazfit Bip 6 Smart Watch 46mm, 14 Day Battery...",4.5 out of 5 stars,$74.99,"Multisport Tracker, Text Messaging, AI Assista...",340 Milliamp Hours,Bluetooth,Built-in GPS,1.97 Inches,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2 Sport Smart Watch Fitness Tra...,4.5 out of 5 stars,$75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2 Sport Smart Watch Fitness Tra...,4.5 out of 5 stars,$75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3 Outdoor Smart Watch 48mm AMOLE...,4.4 out of 5 stars,$189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3 Outdoor Smart Watch 48mm AMOLE...,4.4 out of 5 stars,$189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
...,...,...,...,...,...,...,...,...,...,...,...
537,Xiaomi Smart Band 9 Global Version (2024) 1.62...,4.5 out of 5 stars,$57.99,Heart Rate Monitor,233 Milliamp Hours,Bluetooth,No GPS,1.62 Inches,https://m.media-amazon.com/images/I/61WgY4Yssc...,Xiaomi,[{'review': 'This is the 4th generation of Mi ...
538,"Xiaomi Redmi Watch 5 Smartwatch, 2.07"" LCD Dis...",4.4 out of 5 stars,$139.99,"Voice Call, Lightweight, Activity Tracker, Pho...",N/A,Bluetooth,No GPS,2.07 Inches,https://m.media-amazon.com/images/I/51X9nE628c...,Xiaomi,[{'review': 'Calidad - precio. El producto es ...
539,"Xiaomi Watch 2 Pro, Google apps Directly on Yo...",4.1 out of 5 stars,$259.99,Bluetooth,N/A,Bluetooth,Built-in GPS,1.43 Inches,https://m.media-amazon.com/images/I/81LQkI4eoq...,Xiaomi,"[{'review': 'Me encanta este reloj, cumple con..."
540,"Xiaomi Redmi 5 Lite Smart Watch, Step Counter ...",4.6 out of 5 stars,$52.24,"Multisport Tracker, Activity Tracker, Alarm Cl...",470 Amp Hours,Bluetooth,N/A,2 Inches,https://m.media-amazon.com/images/I/61Mr-G86wZ...,Xiaomi,[{'review': 'Although positioned as an entry-l...


Baca file dengan pandas, kemudian save sebagai csv sebgai backup.

In [58]:
# Save to CSV
df.to_csv('data_before_clean.csv', index=False)

In [59]:
# Data Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             542 non-null    object
 1   rating            542 non-null    object
 2   price             542 non-null    object
 3   features          542 non-null    object
 4   Battery Capacity  542 non-null    object
 5   Connectivity      542 non-null    object
 6   GPS               542 non-null    object
 7   Screen Size       542 non-null    object
 8   img_url           542 non-null    object
 9   Brand             542 non-null    object
 10  Review            542 non-null    object
dtypes: object(11)
memory usage: 46.7+ KB


Nama kolom diubah agar lebih mudah.

In [60]:
# Change Column Name

df.rename(columns={
    'title': 'products',
    'Battery Capacity': 'battery',
    'Connectivity': 'connectivity',
    'GPS': 'gps',
    'Screen Size': 'screen_size',
    'Brand': 'brand',
    'Review': 'reviews'
}, inplace=True)

df.head()


,products,rating,price,features,battery,connectivity,gps,screen_size,img_url,brand,reviews
0,"Amazfit Bip 6 Smart Watch 46mm, 14 Day Battery...",4.5 out of 5 stars,$74.99,"Multisport Tracker, Text Messaging, AI Assista...",340 Milliamp Hours,Bluetooth,Built-in GPS,1.97 Inches,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2 Sport Smart Watch Fitness Tra...,4.5 out of 5 stars,$75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2 Sport Smart Watch Fitness Tra...,4.5 out of 5 stars,$75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3 Outdoor Smart Watch 48mm AMOLE...,4.4 out of 5 stars,$189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3 Outdoor Smart Watch 48mm AMOLE...,4.4 out of 5 stars,$189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."


Memastikan brand yang diinginkan ada pada dataset.

In [61]:
# Check brand
df.brand.unique()

array(['Amazfit', 'Apple Watch', 'Fitbit', 'Garmin', 'Google', 'Huawei',
       'Samsung', 'Xiaomi'], dtype=object)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   products      542 non-null    object
 1   rating        542 non-null    object
 2   price         542 non-null    object
 3   features      542 non-null    object
 4   battery       542 non-null    object
 5   connectivity  542 non-null    object
 6   gps           542 non-null    object
 7   screen_size   542 non-null    object
 8   img_url       542 non-null    object
 9   brand         542 non-null    object
 10  reviews       542 non-null    object
dtypes: object(11)
memory usage: 46.7+ KB


# Column product

In [63]:
# Change 'N/A' to NaN
df['products'] = df['products'].replace('N/A', np.nan)

In [64]:
# Drop Data null in products
df = df.dropna(subset=['products']).reset_index(drop=True)

Drop missing value jika ada.

In [65]:
# Filter products based on Brand

# Create regex pattern with keyword
pattern = r'^(Amazfit|Apple Watch|Fitbit|Garmin|Xiaomi|Huawei Watch|Samsung|Google Pixel)'

# Filter products with keyword product (case-insensitive)
df = df[df['products'].str.contains(pattern, case=False, na=False, regex=True)]

# Reset index
df = df.reset_index(drop=True)


Filtering product yang hanya mencantum brand smart watch yang ditentukan.

In [66]:
# Make sure brand is in product name

df = df[df.apply(lambda row: str(row['brand']).lower() in str(row['products']).lower(), axis=1)]

Memastikan data di brand tercantum di kolom nama produk (products)

In [67]:
# Delete data that have accesorris in products

df = df[~df['products'].str.contains(r'\b(cable|carger)\b', case=False, na=False)]

Menghapus data yang merupkan aksesoris dari smartwatch, bukan smart watch itu sendiri.

In [68]:
# Strip products name after delimiter

def cut_at_first_delimiter(text):
    if not isinstance(text, str):
        return text
    
    delimiters = [',', '，', '(', '[', ' -']
    positions = [text.find(d) for d in delimiters if text.find(d) != -1]
    
    if positions:
        cut_pos = min(positions)
        return text[:cut_pos].strip()
    else:
        return text.strip()

df['products'] = df['products'].apply(cut_at_first_delimiter)


Memotong nama produk sesuai tanda baca, yang biasanya setelahnya merupkan keterangan produk.

In [69]:
# Strip products name after number

def cut_after_last_number(text):
    if not isinstance(text, str):
        return text
    
    match = re.search(r'^(.*?\d+)(?:\s|$)', text)
    
    if match:
        return match.group(1).strip()
    else:
        return text.strip()
    
df['products'] = df['products'].apply(cut_after_last_number)

Memotong nama produk setekah angka, angka biasanya no series produk, setelahnya biasanya merrupkan keterangan produk.

In [70]:
# Strip amount of words

def extract_product_name(text):
    if not isinstance(text, str):
        return text
    
    # Take only 5 words
    return ' '.join(text.split()[:5])

df['products'] = df['products'].apply(extract_product_name)


Membatasi nama produk hanya dalam 5 kata.

In [71]:
# Remove symbol from 'products'
df['products'] = df['products'].str.replace(r'[®™]', '', regex=True).str.strip()

Menghapus symbol-symbol yang berada pada nama produk.

In [72]:
df.products.unique()

array(['Amazfit Bip 6', 'Amazfit Active 2', 'Amazfit T-Rex 3',
       'Amazfit GTR Mini Smart Watch', 'Amazfit Band 7',
       'Amazfit Active Smart Watch 42mm',
       'Amazfit Balance Smart Watch 46mm', 'Amazfit Refurbished BIP 3',
       'Amazfit Bip 5', 'Amazfit GTR 3', 'Amazfit Balance 2',
       'Amazfit Refurbished T-Rex 3', 'Amazfit Renewed Bip 5',
       'Amazfit Bip 3', 'Amazfit GTS 2',
       'Amazfit T-Rex Ultra Tactical Smart', 'Apple Watch Series 10',
       'Apple Watch Ultra 2', 'Apple Watch Series 5',
       'Apple Watch Series 7', 'Apple Watch Series 6',
       'Apple Watch Series 8', 'Apple Watch SE', 'Apple Watch Ultra',
       'Apple Watch SE 2nd Gen', 'Apple Watch Series 9',
       'Apple Watch Series SE 44mm', 'Fitbit Versa 4', 'Fitbit Charge 6',
       'Fitbit Inspire 3', 'Fitbit Sense 2', 'Fitbit Versa 2',
       'Fitbit Google Ace LTE', 'Fitbit Versa Smart Watch',
       'Fitbit Versa Lite Smartwatch',
       'Fitbit Sense Advanced Smartwatch with', 'Google Pi

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 0 to 446
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   products      413 non-null    object
 1   rating        413 non-null    object
 2   price         413 non-null    object
 3   features      413 non-null    object
 4   battery       413 non-null    object
 5   connectivity  413 non-null    object
 6   gps           413 non-null    object
 7   screen_size   413 non-null    object
 8   img_url       413 non-null    object
 9   brand         413 non-null    object
 10  reviews       413 non-null    object
dtypes: object(11)
memory usage: 38.7+ KB


# Column rating

In [74]:
# Clean and Convert rating

def cut_after_space_and_convert(text):
    if not isinstance(text, str):
        return np.nan

    first_part = text.split(' ')[0]
    try:
        return float(first_part)
    except ValueError:
        return np.nan

df['rating'] = df['rating'].apply(cut_after_space_and_convert)

In [75]:
df.head()

,products,rating,price,features,battery,connectivity,gps,screen_size,img_url,brand,reviews
0,Amazfit Bip 6,4.5,$74.99,"Multisport Tracker, Text Messaging, AI Assista...",340 Milliamp Hours,Bluetooth,Built-in GPS,1.97 Inches,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2,4.5,$75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2,4.5,$75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3,4.4,$189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3,4.4,$189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 0 to 446
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      413 non-null    object 
 1   rating        413 non-null    float64
 2   price         413 non-null    object 
 3   features      413 non-null    object 
 4   battery       413 non-null    object 
 5   connectivity  413 non-null    object 
 6   gps           413 non-null    object 
 7   screen_size   413 non-null    object 
 8   img_url       413 non-null    object 
 9   brand         413 non-null    object 
 10  reviews       413 non-null    object 
dtypes: float64(1), object(10)
memory usage: 38.7+ KB


# Column price

In [77]:
# Change 'N/A' to NaN
df['price'] = df['price'].replace('N/A', np.nan)

In [78]:
# Clean and convert price

df.loc[:, 'price'] = pd.to_numeric(
    df['price'].str.replace(r'[\$,]', '', regex=True),
    errors='coerce')

In [79]:
df.head()

,products,rating,price,features,battery,connectivity,gps,screen_size,img_url,brand,reviews
0,Amazfit Bip 6,4.5,74.99,"Multisport Tracker, Text Messaging, AI Assista...",340 Milliamp Hours,Bluetooth,Built-in GPS,1.97 Inches,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 0 to 446
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      413 non-null    object 
 1   rating        413 non-null    float64
 2   price         412 non-null    object 
 3   features      413 non-null    object 
 4   battery       413 non-null    object 
 5   connectivity  413 non-null    object 
 6   gps           413 non-null    object 
 7   screen_size   413 non-null    object 
 8   img_url       413 non-null    object 
 9   brand         413 non-null    object 
 10  reviews       413 non-null    object 
dtypes: float64(1), object(10)
memory usage: 38.7+ KB


# Column features

In [81]:
# Change 'N/A' to NaN
df['features'] = df['features'].replace('N/A', np.nan)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 0 to 446
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      413 non-null    object 
 1   rating        413 non-null    float64
 2   price         412 non-null    object 
 3   features      386 non-null    object 
 4   battery       413 non-null    object 
 5   connectivity  413 non-null    object 
 6   gps           413 non-null    object 
 7   screen_size   413 non-null    object 
 8   img_url       413 non-null    object 
 9   brand         413 non-null    object 
 10  reviews       413 non-null    object 
dtypes: float64(1), object(10)
memory usage: 38.7+ KB


# Column battery

In [83]:
# Clean and Change battery

def clean_battery(value):
    if not isinstance(value, str):
        return np.nan
    
    value = value.strip().lower()
    
    if value == 'n/a':
        return np.nan

    # Handle scientific notation like '1E+1'
    if re.match(r'^\d+(\.\d+)?[eE][+-]?\d+', value):
        try:
            return float(value)
        except:
            return np.nan

    # Extract numeric value
    match = re.search(r'(\d+(\.\d+)?([eE][+-]?\d+)?)', value)
    if not match:
        return np.nan
    
    num = float(match.group(1))

    # Precise unit checks
    if 'milliamp' in value or 'mah' in value:
        return num  # already in mAh
    elif re.search(r'\bamp(?:ere)? hour', value):  # only match 'amp hour' not 'milliamp'
        return num * 1000  # convert to mAh
    elif 'hour' in value or 'day' in value:
        return np.nan  # not a capacity, but a duration
    elif re.fullmatch(r'\d+(\.\d+)?', value):
        return num
    else:
        return np.nan
    
df['battery'] = df['battery'].apply(clean_battery)

In [84]:
df.head()

,products,rating,price,features,battery,connectivity,gps,screen_size,img_url,brand,reviews
0,Amazfit Bip 6,4.5,74.99,"Multisport Tracker, Text Messaging, AI Assista...",340.0,Bluetooth,Built-in GPS,1.97 Inches,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270.0,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270.0,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",NaN,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",NaN,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 0 to 446
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      413 non-null    object 
 1   rating        413 non-null    float64
 2   price         412 non-null    object 
 3   features      386 non-null    object 
 4   battery       191 non-null    float64
 5   connectivity  413 non-null    object 
 6   gps           413 non-null    object 
 7   screen_size   413 non-null    object 
 8   img_url       413 non-null    object 
 9   brand         413 non-null    object 
 10  reviews       413 non-null    object 
dtypes: float64(2), object(9)
memory usage: 38.7+ KB


# Column connectivity

In [86]:
# Change 'N/A' to NaN
df['connectivity'] = df['connectivity'].replace('N/A', np.nan)

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 0 to 446
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      413 non-null    object 
 1   rating        413 non-null    float64
 2   price         412 non-null    object 
 3   features      386 non-null    object 
 4   battery       191 non-null    float64
 5   connectivity  387 non-null    object 
 6   gps           413 non-null    object 
 7   screen_size   413 non-null    object 
 8   img_url       413 non-null    object 
 9   brand         413 non-null    object 
 10  reviews       413 non-null    object 
dtypes: float64(2), object(9)
memory usage: 38.7+ KB


# Column gps

In [88]:
# Change 'N/A' to NaN
df['gps'] = df['gps'].replace('N/A', np.nan)

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 0 to 446
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      413 non-null    object 
 1   rating        413 non-null    float64
 2   price         412 non-null    object 
 3   features      386 non-null    object 
 4   battery       191 non-null    float64
 5   connectivity  387 non-null    object 
 6   gps           354 non-null    object 
 7   screen_size   413 non-null    object 
 8   img_url       413 non-null    object 
 9   brand         413 non-null    object 
 10  reviews       413 non-null    object 
dtypes: float64(2), object(9)
memory usage: 38.7+ KB


# Column screen_size

In [90]:
# Clean and Convert screen_size

def clean_screen_size(value):

    if not isinstance(value, str):
        return np.nan

    value = value.strip().lower()

    if value in ['n/a', 'na', '']:
        return np.nan

    # Extract numeric value (float or scientific)
    match = re.search(r'(\d+(\.\d+)?([eE][+-]?\d+)?)', value)
    if not match:
        return np.nan

    size = float(match.group(1))

    # Convert based on unit
    if 'millimeter' in value:
        return size / 25.4  # 1 inch = 25.4 mm
    elif 'centimeter' in value:
        return size / 2.54  # 1 inch = 2.54 cm
    elif 'meter' in value:
        return size * 39.37  # probably error, but convert anyway
    else:
        return size  # assume already in inches
    
df['screen_size'] = df['screen_size'].apply(clean_screen_size)

In [91]:
df.head()

,products,rating,price,features,battery,connectivity,gps,screen_size,img_url,brand,reviews
0,Amazfit Bip 6,4.5,74.99,"Multisport Tracker, Text Messaging, AI Assista...",340.0,Bluetooth,Built-in GPS,1.97,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270.0,Bluetooth,Built-in GPS,1.32,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270.0,Bluetooth,Built-in GPS,1.32,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",NaN,"Bluetooth, Wi-Fi",Built-in GPS,1.50,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",NaN,"Bluetooth, Wi-Fi",Built-in GPS,1.50,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 0 to 446
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      413 non-null    object 
 1   rating        413 non-null    float64
 2   price         412 non-null    object 
 3   features      386 non-null    object 
 4   battery       191 non-null    float64
 5   connectivity  387 non-null    object 
 6   gps           354 non-null    object 
 7   screen_size   400 non-null    float64
 8   img_url       413 non-null    object 
 9   brand         413 non-null    object 
 10  reviews       413 non-null    object 
dtypes: float64(3), object(8)
memory usage: 38.7+ KB


# Column img_url

In [93]:
# Change 'N/A' to NaN
df['img_url'] = df['img_url'].replace('N/A', np.nan)

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 0 to 446
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      413 non-null    object 
 1   rating        413 non-null    float64
 2   price         412 non-null    object 
 3   features      386 non-null    object 
 4   battery       191 non-null    float64
 5   connectivity  387 non-null    object 
 6   gps           354 non-null    object 
 7   screen_size   400 non-null    float64
 8   img_url       413 non-null    object 
 9   brand         413 non-null    object 
 10  reviews       413 non-null    object 
dtypes: float64(3), object(8)
memory usage: 38.7+ KB


# Split data

In [95]:
# Save before split

df.to_csv('data_before_split.csv', index=False)

Data Product

In [96]:
# Data product

df_product = df.drop('reviews', axis=1)
df_product

,products,rating,price,features,battery,connectivity,gps,screen_size,img_url,brand
0,Amazfit Bip 6,4.5,74.99,"Multisport Tracker, Text Messaging, AI Assista...",340.0,Bluetooth,Built-in GPS,1.97,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit
1,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270.0,Bluetooth,Built-in GPS,1.32,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit
2,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270.0,Bluetooth,Built-in GPS,1.32,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit
3,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",NaN,"Bluetooth, Wi-Fi",Built-in GPS,1.50,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit
4,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",NaN,"Bluetooth, Wi-Fi",Built-in GPS,1.50,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit
...,...,...,...,...,...,...,...,...,...,...
442,Xiaomi Smart Band 9,4.5,57.99,Heart Rate Monitor,233.0,Bluetooth,No GPS,1.62,https://m.media-amazon.com/images/I/61WgY4Yssc...,Xiaomi
443,Xiaomi Redmi Watch 5,4.4,139.99,"Voice Call, Lightweight, Activity Tracker, Pho...",NaN,Bluetooth,No GPS,2.07,https://m.media-amazon.com/images/I/51X9nE628c...,Xiaomi
444,Xiaomi Watch 2,4.1,259.99,Bluetooth,NaN,Bluetooth,Built-in GPS,1.43,https://m.media-amazon.com/images/I/81LQkI4eoq...,Xiaomi
445,Xiaomi Redmi 5,4.6,52.24,"Multisport Tracker, Activity Tracker, Alarm Cl...",470000.0,Bluetooth,NaN,2.00,https://m.media-amazon.com/images/I/61Mr-G86wZ...,Xiaomi


In [97]:
# Save data_product

df_product.to_csv('data_product.csv', index=False)

Data Review

In [98]:
# Data review

df_review = df[['products', 'reviews']]
df_review

,products,reviews
0,Amazfit Bip 6,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3,"[{'review': 'So far, after using the watch for..."
...,...,...
442,Xiaomi Smart Band 9,[{'review': 'This is the 4th generation of Mi ...
443,Xiaomi Redmi Watch 5,[{'review': 'Calidad - precio. El producto es ...
444,Xiaomi Watch 2,"[{'review': 'Me encanta este reloj, cumple con..."
445,Xiaomi Redmi 5,[{'review': 'Although positioned as an entry-l...


In [99]:
# Split data inside reviews column

# Make sure data is list of dict
def parse_review_column(val):
    try:
        return ast.literal_eval(val) if isinstance(val, str) else val
    except:
        return np.nan

df_review['reviews'] = df_review['reviews'].apply(parse_review_column)

# Explode
df_exploded = df_review.explode('reviews').reset_index(drop=True)

# Take 'review' dan 'rating' from dict
df_exploded['individual_review'] = df_exploded['reviews'].apply(lambda x: x.get('review') if isinstance(x, dict) else np.nan)
df_exploded['individual_rating'] = df_exploded['reviews'].apply(lambda x: x.get('rating') if isinstance(x, dict) else np.nan)

# delete previous reviews column
df_exploded = df_exploded.drop(columns=['reviews'])

# Show result
df_exploded


,products,individual_review,individual_rating
0,Amazfit Bip 6,I purchased this watch to replace a dead Samsu...,5.0 out of 5 stars
1,Amazfit Bip 6,"Pleasantly surprised. Finally, a less expensiv...",5.0 out of 5 stars
2,Amazfit Bip 6,If you’re looking to purchase a smart watch th...,5.0 out of 5 stars
3,Amazfit Bip 6,"This watch does well, considered the price. Th...",4.0 out of 5 stars
4,Amazfit Bip 6,This watch is amazing quality for the price! I...,5.0 out of 5 stars
...,...,...,...
2788,Xiaomi Redmi Watch 4,Awsome product... Good price and reabilty,5.0 out of 5 stars
2789,Xiaomi Redmi Watch 4,"My husband used it after he got it, very relia...",5.0 out of 5 stars
2790,Xiaomi Redmi Watch 4,Me encanta,5.0 out of 5 stars
2791,Xiaomi Redmi Watch 4,,5.0 out of 5 stars


In [100]:
# Clean and Convert individual_rating

def cut_after_space_and_convert(text):
    if not isinstance(text, str):
        return np.nan

    first_part = text.split(' ')[0]  # get substring before first space
    try:
        return float(first_part)
    except ValueError:
        return np.nan

df_exploded['individual_rating'] = df_exploded['individual_rating'].apply(cut_after_space_and_convert)

In [101]:
df_exploded

,products,individual_review,individual_rating
0,Amazfit Bip 6,I purchased this watch to replace a dead Samsu...,5.0
1,Amazfit Bip 6,"Pleasantly surprised. Finally, a less expensiv...",5.0
2,Amazfit Bip 6,If you’re looking to purchase a smart watch th...,5.0
3,Amazfit Bip 6,"This watch does well, considered the price. Th...",4.0
4,Amazfit Bip 6,This watch is amazing quality for the price! I...,5.0
...,...,...,...
2788,Xiaomi Redmi Watch 4,Awsome product... Good price and reabilty,5.0
2789,Xiaomi Redmi Watch 4,"My husband used it after he got it, very relia...",5.0
2790,Xiaomi Redmi Watch 4,Me encanta,5.0
2791,Xiaomi Redmi Watch 4,,5.0


In [102]:
# Keep individual_review in English only

DetectorFactory.seed = 8

def is_english(text):
    if not isinstance(text, str) or text.strip() == '':
        return False
    try:
        return detect(text) == 'en'
    except:
        return False

# apply filter
df_exploded['is_english'] = df_exploded['individual_review'].apply(is_english)

# Drop that is not English
df_reviews_en = df_exploded[df_exploded['is_english']].drop(columns=['is_english']).reset_index(drop=True)

In [103]:
df_reviews_en

,products,individual_review,individual_rating
0,Amazfit Bip 6,I purchased this watch to replace a dead Samsu...,5.0
1,Amazfit Bip 6,"Pleasantly surprised. Finally, a less expensiv...",5.0
2,Amazfit Bip 6,If you’re looking to purchase a smart watch th...,5.0
3,Amazfit Bip 6,"This watch does well, considered the price. Th...",4.0
4,Amazfit Bip 6,This watch is amazing quality for the price! I...,5.0
...,...,...,...
2539,Xiaomi Redmi Watch 4,Great watch for working out and keeping track ...,5.0
2540,Xiaomi Redmi Watch 4,I have had a number of smartwatches including ...,4.0
2541,Xiaomi Redmi Watch 4,Awsome product... Good price and reabilty,5.0
2542,Xiaomi Redmi Watch 4,"My husband used it after he got it, very relia...",5.0


In [104]:
df_reviews_en.duplicated().sum()

1255

In [105]:
df_reviews_en = df_reviews_en.drop_duplicates()

In [106]:
# Check if there is 'N/A'

print(df_reviews_en['individual_review'].str.contains('N/A', na=False).any())

False


In [107]:
df_reviews_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1289 entries, 0 to 2543
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   products           1289 non-null   object 
 1   individual_review  1289 non-null   object 
 2   individual_rating  1289 non-null   float64
dtypes: float64(1), object(2)
memory usage: 40.3+ KB


In [108]:
# Save data_product

df_reviews_en.to_csv('data_review.csv', index=False)